In [3]:
import nltk # Natural Language Toolkit
nltk.download('stopwords') #rude words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# dataset - цель научить машину идентифицировать авторов по их стилю 
import numpy as np
import pandas as pd 

import sklearn
from sklearn.pipeline import FeatureUnion # delete rude words
from sklearn.model_selection import train_test_split

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [17]:
df=pd.read_csv(r'pipeline-author-identification.csv',header=0)
df=df.dropna() #Удаление пропущенных данных
print(df.shape) #(19579 - строк, 9 - столбцов)
df.set_index('id',inplace= True)
df.head()

(19579, 9)


,text,author,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,,,
id26305,"This process, however, afforded me no means of...",EAP,this process however afforded me no means of a...,224,41,21,6.380952,4
id17569,It never once occurred to me that the fumbling...,HPL,it never once occurred to me that the fumbling...,70,14,6,6.166667,0
id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...,195,36,19,5.947368,4
id27763,How lovely is spring As we looked from Windsor...,MWS,how lovely is spring as we looked from windsor...,202,34,21,6.476190,3
id12958,"Finding nothing else, not even gold, the Super...",HPL,finding nothing else not even gold the superin...,170,27,16,7.187500,2


In [18]:
features = [c for c in df.columns.values if c not in ['id', 'text', 'author']] #not numeric values
numeric_features = [c for c in df.columns.values if c not in ['id', 'text', 'author', 'processed']]

target = 'author'

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size = 0.33, random_state=22)
X_train.head()

,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,
id22174,she was intoning some croaking ritual in a lan...,154,24,13,7.461538,1
id16339,there are some subjects upon which i take plea...,66,12,5,6.000000,0
id17030,the minister i believe has written learnedly o...,73,11,6,8.500000,0
id07743,the sound came from a squat towered stone chur...,199,33,19,6.684211,1
id19457,even as she spoke i drew near to her as if in ...,122,28,10,4.900000,2
